# Datenbanksetup

Das Notebook dient dazu, das Schema für die GraphDB zu erstellen und die Spielwelt zu generieren.

In [ ]:
import os
from dotenv import load_dotenv
from neo4j import GraphDatabase
from sentence_transformers import SentenceTransformer, util

load_dotenv(dotenv_path='../.env')


In [ ]:
driver = GraphDatabase.driver(
    uri=os.getenv('NEO4J_URI'),
    auth=(
        os.getenv('NEO4J_USER'), 
        os.getenv('NEO4J_PASSWORD')
    )
)

print('Docker starten ;-)')
print(f'URI: {os.getenv("NEO4J_URI")}')
print(f'UI:  http://localhost:7474')

# Queryhelper
def run_query(query, params=None):
    
    # Ergebnisliste
    results = []

    # Session öffnen
    with driver.session() as session:

        # Query
        result = session.run(query, params or {})

        # Ergebnisse wegspeichern
        records = [r.data() for r in result]
        results.extend(records)

    return results

In [ ]:
# WARNUNG: Löscht ALLE Daten!
# Uncomment nur wenn du wirklich resetten willst:

def reset_db():
    with driver.session() as session:
        session.run("MATCH (n) DETACH DELETE n")
        print("✓ Database reset - alle Nodes gelöscht")

reset_db()

In [ ]:
emb_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

## Constrains und Index anlegen

In [ ]:
# Constrains festlegen
constrain_queries = [
    'CREATE CONSTRAINT location_id IF NOT EXISTS FOR (l:Location) REQUIRE l.id IS UNIQUE',
    'CREATE CONSTRAINT item_id     IF NOT EXISTS FOR (i:Item)     REQUIRE i.id IS UNIQUE',
    'CREATE CONSTRAINT npc_id      IF NOT EXISTS FOR (n:NPC)      REQUIRE n.id IS UNIQUE',
    'CREATE CONSTRAINT player_id   IF NOT EXISTS FOR (p:Player)   REQUIRE p.id IS UNIQUE',
]

[run_query(query) for query in constrain_queries]
print("✓ Constraints erstellt")

# Index festlegen
entity_indexes = [
    "CREATE INDEX location_name IF NOT EXISTS FOR (l:Location) ON (l.name)",
    "CREATE INDEX item_name     IF NOT EXISTS FOR (i:Item)     ON (i.name)"
]

[run_query(query) for query in entity_indexes]
print("✓ Property Indexes erstellt")

# Vector Indexes für Semantic Search
# Dimension: 384 (paraphrase-multilingual-MiniLM-L12-v2)
vector_indexes = [
    """
    CREATE VECTOR INDEX item_name_index IF NOT EXISTS
    FOR (i:Item) ON i.name_emb
    OPTIONS {indexConfig: {
      `vector.dimensions`: 384,
      `vector.similarity_function`: 'cosine'
    }}
    """,
    """
    CREATE VECTOR INDEX location_name_index IF NOT EXISTS
    FOR (l:Location) ON l.name_emb
    OPTIONS {indexConfig: {
      `vector.dimensions`: 384,
      `vector.similarity_function`: 'cosine'
    }}
    """,
    """
    CREATE VECTOR INDEX persons_name_index IF NOT EXISTS
    FOR (p:Persons) ON p.name_emb
    OPTIONS {indexConfig: {
      `vector.dimensions`: 384,
      `vector.similarity_function`: 'cosine'
    }}
    """
]

for query in vector_indexes:
    run_query(query)
    
print("✓ Vector Indexes erstellt (384 dim, cosine similarity)")

## Objekte erstellen

In [ ]:
# Orte/Räume: id, name, beschreibung
locations_data = [
    {
        'id': 'taverne',
        'name': 'Mo\'s Taverne',
        'description': 'Eine alte Taverne, etwas heruntergekommen aber bleibt. Hier wird jeden Abend gefeiert.',
    },
    {
        'id': 'marktplatz',
        'name': 'Marktplatz',
        'description': 'Das ist der Markplatz des kleinen, beschaulichen Ortes.'
    },
    {
        'id': 'finsterwald',
        'name': 'Finsterwald',
        'description': 'in dunkler, übelriechender Wald voller Geister und anderen Ängsten.'
    },
    {
        'id': 'schmiede',
        'name': 'Alte Schmiede',
        'description': 'Eine verlassene Schmiede am Rande des Dorfes. Der Amboss steht noch, aber das Feuer ist längst erloschen.'
    },

]


for location in locations_data:

    query = """
    MERGE (l:Location {id: $id})
    ON CREATE SET 
        l.name = $name, 
        l.description = $description,
        l.name_emb = $name_emb,
        l.description_emb = $description_emb
    RETURN l
    """

    name_emb = emb_model.encode(location['name'])
    description_emb = emb_model.encode(location['description'])

    params = {
        'id': location['id'],
        'name': location['name'],
        'description': location['description'],
        'name_emb': name_emb,
        'description_emb': description_emb
    }

    run_query(query, params)

In [ ]:
# Items: id, name, beschreibung
items_data = [
    {
        'id': 'truhe',
        'name': 'Alte Truhe',
        'description': 'Eine alte Truhe mit einem rostigen Schloss. Sie steht schon länger hier. Alt und verwittert.'
    },
    {
        'id': 'schluessel',
        'name': 'Rostiger Schlüssel',
        'description': 'Ein rostiger Schlüssel der nur in ein ganz bestimmtes Schloss passt.'
    },
    {
        'id': 'fackel',
        'name': 'Flackernde Fackel',
        'description': 'Eine alte Fackel. Noch nicht angezündet, aber bereit Licht zu spenden.',
        'is_lit': False  # Zustand: noch nicht angezündet
    },
    {
        'id': 'streichhoelzer',
        'name': 'Streichhölzer',
        'description': 'Eine kleine Schachtel mit Streichhölzern. Könnten nützlich sein.'
    },
    {
        'id': 'hammer',
        'name': 'Schwerer Hammer',
        'description': 'Ein Schmiedehammer mit massivem Kopf. Schwer aber wirksam.'
    },
    {
        'id': 'beutel',
        'name': 'Lederbeutel',
        'description': 'Ein kleiner Lederbeutel. Es klappert etwas darin.'
    },
    {
        'id': 'schwert',
        'name': 'Altes Schwert',
        'description': 'Ein altes Schwert, das an der Wand der Taverne hängt. Rostig aber noch scharf.'
    },
    {
        'id': 'buch',
        'name': 'Vergilbtes Buch',
        'description': 'Ein altes Buch mit vergilbten Seiten. Scheint ein Rezeptbuch zu sein.'
    }
]


for item in items_data:

    query = """
    MERGE (i:Item {id: $id})
    ON CREATE SET 
        i.name = $name, 
        i.description = $description,
        i.name_emb = $name_emb,
        i.description_emb = $description_emb,
        i.is_lit = $is_lit
    RETURN i
    """

    name_emb = emb_model.encode(item['name'])
    description_emb = emb_model.encode(item['description'])

    params = {
        'id': item['id'],
        'name': item['name'],
        'description': item['description'],
        'name_emb': name_emb,
        'description_emb': description_emb,
        'is_lit': item.get('is_lit', None)  # Nur für Items mit Zustand
    }

    run_query(query, params)

In [ ]:
# NPCs: id, name, beschreibung
persons_data = [
    {
        'id': 'wirt',
        'name': 'Schenk',
        'description': 'Ein alter, grummiger Wirt, der seinen Gästen stets zu wenig einschenkt.'
    },
    {
        'id': 'player',
        'name': 'Player',
        'description': 'Hier könnte Dein Name stehen!'
    },
    {
        'id': 'haendler',
        'name': 'Wanderhändler',
        'description': 'Ein freundlicher Händler mit einem großen Rucksack voller Waren.'
    }
]

for person in persons_data:

    query = """
    MERGE (p:Persons {id: $id})
    ON CREATE SET 
        p.name = $name, 
        p.description = $description,
        p.name_emb = $name_emb,
        p.description_emb = $description_emb
    RETURN p
    """
    
    name_emb = emb_model.encode(person['name'])
    description_emb = emb_model.encode(person['description'])

    params = {
        'id': person['id'],
        'name': person['name'],
        'description': person['description'],
        'name_emb': name_emb,
        'description_emb': description_emb
    }

    run_query(query, params)

## Placing aller Objekte

In [ ]:
# Persons start
player_edges = [
    """
    MATCH (start:Persons {id: 'player'}), (ende:Location {id: 'marktplatz'})
    MERGE (start)-[:IST_IN]->(ende)
    RETURN start, ende
    """,
    """
    MATCH (start:Persons {id: 'wirt'}), (ende:Location {id: 'taverne'})
    MERGE (start)-[:IST_IN]->(ende)
    RETURN start, ende
    """,
    """
    MATCH (start:Persons {id: 'haendler'}), (ende:Location {id: 'marktplatz'})
    MERGE (start)-[:IST_IN]->(ende)
    RETURN start, ende
    """
]

[run_query(query) for query in player_edges]

In [ ]:
# Wege
location_edges = [
    """
    MATCH
        (m:Location {id: 'marktplatz'}),
        (t:Location {id: 'taverne'}),
        (f:Location {id: 'finsterwald'}),
        (s:Location {id: 'schmiede'})
    MERGE (m)-[:ERREICHT]->(t)
    MERGE (t)-[:ERREICHT]->(m)
    MERGE (m)-[:ERREICHT]->(f)
    MERGE (f)-[:ERREICHT]->(m)
    MERGE (m)-[:ERREICHT]->(s)
    MERGE (s)-[:ERREICHT]->(m)
    """
]

[run_query(query) for query in location_edges]

In [ ]:
# Items platzieren
item_edges = [
    """
    MATCH
        (ta:Location {id: 'taverne'}),
        (f:Location {id: 'finsterwald'}),
        (s:Location {id: 'schmiede'}),
        (m:Location {id: 'marktplatz'}),
        (schluessel:Item {id: 'schluessel'}),
        (truhe:Item {id: 'truhe'}),
        (fackel:Item {id: 'fackel'}),
        (streichhoelzer:Item {id: 'streichhoelzer'}),
        (hammer:Item {id: 'hammer'}),
        (beutel:Item {id: 'beutel'}),
        (schwert:Item {id: 'schwert'}),
        (buch:Item {id: 'buch'}),
        (wirt:Persons {id: 'wirt'})
    MERGE (schluessel)-[:IST_IN]->(ta)
    MERGE (truhe)-[:IST_IN]->(f)
    MERGE (fackel)-[:IST_IN]->(s)
    MERGE (streichhoelzer)-[:TRÄGT]->(wirt)
    MERGE (hammer)-[:IST_IN]->(s)
    MERGE (beutel)-[:IST_IN]->(m)
    MERGE (schwert)-[:IST_IN]->(ta)
    MERGE (buch)-[:IST_IN]->(ta)
    """
]

[run_query(query) for query in item_edges]

In [ ]:
# Quests & Item-Interaktionen
quest_edges = [
    # Truhe öffnen: Mit Schlüssel ODER mit Hammer zerbrechen
    """
    MATCH
        (s:Item {id: 'schluessel'}),
        (t:Item {id: 'truhe'})
    MERGE (s)-[:OEFFNET]->(t)
    """,
    """
    MATCH
        (h:Item {id: 'hammer'}),
        (t:Item {id: 'truhe'})
    MERGE (h)-[:KANN_ZERBRECHEN]->(t)
    """,
    # Fackel-Quest: Streichhölzer zünden Fackel an
    """
    MATCH
        (st:Item {id: 'streichhoelzer'}),
        (f:Item {id: 'fackel'})
    MERGE (st)-[:KANN_ANZÜNDEN]->(f)
    """,
    # Angezündete Fackel beleuchtet Finsterwald
    """
    MATCH
        (f:Item {id: 'fackel'}),
        (fw:Location {id: 'finsterwald'})
    MERGE (f)-[:BELEUCHTET]->(fw)
    """
]

[run_query(query) for query in quest_edges]

## Testwelt für Smart Parser

Minimale Testwelt für Parser-Tests (komplett isoliert)

In [ ]:
# Test-Location
test_locations_data = [
    {
        'id': 'testroom',
        'name': 'Testraum',
        'description': 'Ein Raum zum Testen der Parser-Funktionen.'
    }
]

for location in test_locations_data:
    query = """
    MERGE (l:Location {id: $id})
    ON CREATE SET 
        l.name = $name, 
        l.description = $description,
        l.name_emb = $name_emb,
        l.description_emb = $description_emb
    RETURN l
    """
    
    name_emb = emb_model.encode(location['name'])
    description_emb = emb_model.encode(location['description'])
    
    params = {
        'id': location['id'],
        'name': location['name'],
        'description': location['description'],
        'name_emb': name_emb,
        'description_emb': description_emb
    }
    
    run_query(query, params)

In [ ]:
# Test-Items mit separaten Embeddings (Name, Synonyme, Beschreibung)
test_items_data = [
    {
        'id': 'test_schluessel',
        'name': 'Goldener Schlüssel',
        'description': 'Ein glänzender goldener Schlüssel',
        'synonyms': ['Schlüssel', 'goldener Schlüssel', 'Dietrich', 'Schloss-Öffner']
    },
    {
        'id': 'test_schwert',
        'name': 'Rostiges Schwert',
        'description': 'Ein altes Schwert mit Rost',
        'synonyms': ['Schwert', 'rostiges Schwert', 'Klinge', 'Waffe', 'Eisenschwert']
    },
    {
        'id': 'test_kristall',
        'name': 'Kristall',
        'description': 'Ein leuchtender Kristall',
        'synonyms': ['Kristall', 'Edelstein', 'Stein', 'funkelnder Brocken', 'glänzender Stein']
    },
    {
        'id': 'test_laterne',
        'name': 'Laterne',
        'description': 'Eine alte flackernde Laterne',
        'synonyms': ['Laterne', 'Lampe', 'Leuchte', 'Licht']
    },
    {
        'id': 'test_dolch',
        'name': 'Dolch',
        'description': 'Ein scharfer Dolch',
        'synonyms': ['Dolch', 'Messer', 'Klinge', 'Waffe']
    },
    {
        'id': 'test_hammer',
        'name': 'Hammer',
        'description': 'Ein rostiger Schmiedehammer',
        'synonyms': ['Hammer', 'Werkzeug', 'Schmiedehammer', 'schweres Ding']
    }
]

for item in test_items_data:
    query = """
    MERGE (i:Item {id: $id})
    ON CREATE SET 
        i.name = $name, 
        i.description = $description,
        i.name_emb = $name_emb,
        i.synonyms_emb = $synonyms_emb,
        i.description_emb = $description_emb
    RETURN i
    """
    
    # Drei separate Embeddings erstellen
    name_emb = emb_model.encode(item['name'])
    synonyms_emb = emb_model.encode(' '.join(item['synonyms']))
    description_emb = emb_model.encode(item['description'])
    
    params = {
        'id': item['id'],
        'name': item['name'],
        'description': item['description'],
        'name_emb': name_emb,
        'synonyms_emb': synonyms_emb,
        'description_emb': description_emb
    }
    
    run_query(query, params)

In [ ]:
# Test-Player
test_persons_data = [
    {
        'id': 'testplayer',
        'name': 'Testplayer',
        'description': 'Ein Spieler zum Testen'
    }
]

for person in test_persons_data:
    query = """
    MERGE (p:Player {id: $id})
    ON CREATE SET 
        p.name = $name, 
        p.description = $description
    RETURN p
    """
    
    params = {
        'id': person['id'],
        'name': person['name'],
        'description': person['description']
    }
    
    run_query(query, params)

In [ ]:
# Test-Player Placement
test_player_edges = [
    """
    MATCH (start:Player {id: 'testplayer'}), (ende:Location {id: 'testroom'})
    MERGE (start)-[:IST_IN]->(ende)
    RETURN start, ende
    """
]

[run_query(query) for query in test_player_edges]

In [ ]:
# Test-Items Placement
test_item_edges = [
    """
    MATCH
        (loc:Location {id: 'testroom'}),
        (schluessel:Item {id: 'test_schluessel'}),
        (schwert:Item {id: 'test_schwert'}),
        (kristall:Item {id: 'test_kristall'}),
        (laterne:Item {id: 'test_laterne'}),
        (dolch:Item {id: 'test_dolch'}),
        (hammer:Item {id: 'test_hammer'})
    MERGE (schluessel)-[:IST_IN]->(loc)
    MERGE (schwert)-[:IST_IN]->(loc)
    MERGE (kristall)-[:IST_IN]->(loc)
    MERGE (laterne)-[:IST_IN]->(loc)
    MERGE (dolch)-[:IST_IN]->(loc)
    MERGE (hammer)-[:IST_IN]->(loc)
    """
]

[run_query(query) for query in test_item_edges]